In [1]:
import json

pp_2_ev_path = "/home/squirrel/ccg-new/projects/perspective/data/pilot3_twowingos/pp_2_ev_20181105.json" 
ev_2_pp_path = "/home/squirrel/ccg-new/projects/perspective/data/pilot3_twowingos/ev_2_pp_20181105.json" 

with open(pp_2_ev_path) as fin, open(ev_2_pp_path) as fin2:
    pp_2_ev = json.load(fin)
    ev_2_pp = json.load(fin2)

In [18]:
import pandas as pd

evidence_rel_path = "/home/squirrel/ccg-new/projects/perspective/data/database_output/webapp_evidencerelation.csv"
evidence_path = "/home/squirrel/ccg-new/projects/perspective/data/database_output/webapp_evidence.csv"
persp_path = "/home/squirrel/ccg-new/projects/perspective/data/database_output/webapp_perspective.csv"

df = pd.read_csv(evidence_rel_path)
df.info()

edf = pd.read_csv(evidence_path)
pdf = pd.read_csv(persp_path)

def get_gold_persp(evidence_id):
    q = df.loc[df.evidence_id == evidence_id]
    if len(q) > 0:
        return q.iloc[0].perspective_id
    else:
        return None
    
def get_gold_evidence(perspective_id):
    q = df.loc[df.perspective_id == perspective_id]
    if len(q) > 0:
        return q.iloc[0].evidence_id
    else:
        return None

def get_persp_text(persp_id):
    q = pdf.loc[pdf.id == persp_id]
    if len(q) > 0:
        return q.iloc[0].title
    else:
        return None
    
def get_evidence_text(evidence_id):
    q = edf.loc[edf.id == evidence_id]
    if len(q) > 0:
        return q.iloc[0].content
    else:
        return None
    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8122 entries, 0 to 8121
Data columns (total 5 columns):
id                8122 non-null int64
author            8122 non-null object
perspective_id    8122 non-null int64
evidence_id       8122 non-null int64
comment           0 non-null float64
dtypes: float64(1), int64(3), object(1)
memory usage: 317.3+ KB


In [3]:
# For each evidence, measure top k accuracy of perspective candidates
k = [1, 10, 20]
correct = [0 for _k in k]

for e2p in ev_2_pp:
    gold_pp_id = get_gold_persp(e2p['ev_id'])
    for idx, _k in enumerate(k):
        if gold_pp_id in e2p['pp_id_list'][:_k]:
            correct[idx] += 1

total = len(ev_2_pp)


for idx, _k in enumerate(k):
    print("Top {} accuracy: {}".format(_k, correct[idx] / total))

Top 1 accuracy: 0.003324304358532381
Top 10 accuracy: 0.07645900024624477
Top 20 accuracy: 0.16670770746121644


In [12]:
# Measure average rank of perspective candidates for every evidence (rank starts from 1)
total = len(ev_2_pp)
total_rank = 0

for e2p in ev_2_pp:
    gold_pp_id = get_gold_persp(e2p['ev_id'])
    if gold_pp_id:
        try:
            total_rank += e2p['pp_id_list'].index(gold_pp_id) + 1
        except ValueError:
            continue
            
print("Mean Rank of evidence candidates = {}".format(total_rank / total))

Mean Rank of evidence candidates = 293.30349667569567


In [20]:
# Hmm things look bad.. Let's look at some specific examples.

# First, worst cases -- check some examples with rank > 1000
rank_th = 2000
example_count = 5
k = 5

for e2p in ev_2_pp:
    gold_pp_id = get_gold_persp(e2p['ev_id'])
    if gold_pp_id:
        try:
            if e2p['pp_id_list'].index(gold_pp_id) >= rank_th:
                print("evidence_id =  {}".format(e2p['ev_id']))
                print("evidence_text =  {}".format(get_evidence_text(e2p['ev_id'])))
                print("gold_persp_id =  {}".format(gold_pp_id))
                print("gold_persp_text =  {}".format(get_persp_text(gold_pp_id)))
                      
                for pid in e2p['pp_id_list'][:k]:
                    print("persp_cand_id =  {}".format(pid))
                    print("persp_cand_text =  {}".format(get_persp_text(pid)))
                
                example_count -= 1
                if example_count <= 0:
                    break
        except ValueError:
            continue
            
        

evidence_id =  44
evidence_text =  Cigarettes are so common that there is hardly any chance all the people will stop. What will happen is that policies, regarding tobacco regulation or banning will mainly restrict the possibilities of the poor. In 2009, in the US, a law to triple the federal excise tax on cigarettes was signed, which meant that the federal tax on cigarette jumped from 39 cents per pack to $1.01 today. The administration projects, that such a "sin tax" will bring in at least $38 billion over the next five years. Smokers, usually coming from lower socio-economic backgrounds (getting welfare, unemployment or disability checks instead of paychecks) still pay the whole cigarette tax, while they do not get the same amount of funding as others. Anyone concerned about widening income inequality should have second thoughts about this distribution of the tax burden1. Effectively this means, that while a higher financial burden might not cause problems to high and middle class sm

In [22]:
# Next, best cases -- check some examples with rank < 10
rank_th = 5
example_count = 5
k = 5

for e2p in ev_2_pp:
    gold_pp_id = get_gold_persp(e2p['ev_id'])
    if gold_pp_id:
        try:
            if e2p['pp_id_list'].index(gold_pp_id) <= rank_th:
                print("evidence_id =  {}".format(e2p['ev_id']))
                print("evidence_text =  {}".format(get_evidence_text(e2p['ev_id'])))
                print("gold_persp_id =  {}".format(gold_pp_id))
                print("gold_persp_text =  {}".format(get_persp_text(gold_pp_id)))
                      
                for pid in e2p['pp_id_list'][:k]:
                    print("persp_cand_id =  {}".format(pid))
                    print("persp_cand_text =  {}".format(get_persp_text(pid)))
                
                example_count -= 1
                if example_count <= 0:
                    break
        except ValueError:
            continue

evidence_id =  235
evidence_text =  While the new ‘EU High Representative for Foreign and Security Policy’ marks only a bold first step towards a more unified voice for the EU, the decisions are indeed still based on a state by state consultation mechanism – hence the name representative. This should however not to be downplayed as a less significant change in how the EU approaches its foreign policy. The consultation aspect is in fact essential to reaching agreement and the importance of not only presenting a united front to the rest of the world (the EU is exemplary in trade policy and environmental policy, but less important when it comes to presenting a united voice in foreign policy as Belgian Foreign minister Mark Eyskens put it in 1991 “Europe is an economic giant, a political dwarf, and a military worm” 1, but also creating a united front through collaboration and debate. One should thus see this not only as a means to an end, but rather as an important mechanism in itself, whe

evidence_id =  940
evidence_text =  Chess is among the sports with the greatest number of federations and of active participants worldwide.12 Its large fan base, however, is completely unrepresented in the world’s largest sports competition. Chess, furthermore, is highly viewer friendly, so its popularity would only flourish from its representation. It is possible to broadcast any game online, for all to watch at home. Games also often feature high commentary action explaining and analysing the players’ strategies, to make them approachable and exciting to the public. Furthermore, it could take the shape of a team sport in the Olympics, leaving greater space for tactics and discouraging draws, once again making it more appealing for a broad audience. [12] “FIDE – World Chess Federation”,  FIDE,  8 April 2009.  http://www.fide.com/fide.html
gold_persp_id =  943
gold_persp_text =  Chess is highly popular and should be represented
persp_cand_id =  943
persp_cand_text =  Chess is highly po

In [5]:
# Let's see if things are better if we do it the other way around -- find evidences candidates given perspectives

# For each perspective, measure top k accuracy of evidence candidates
k = [1, 10, 20]
correct = [0 for _k in k]

for p2e in pp_2_ev:
    gold_ev_id = get_gold_evidence(p2e['pp_id'])
    for idx, _k in enumerate(k):
        if gold_ev_id in p2e['ev_id_list'][:_k]:
            correct[idx] += 1

total = len(pp_2_ev)


for idx, _k in enumerate(k):
    print("Top {} accuracy: {}".format(_k, correct[idx] / total))

Top 1 accuracy: 0.08224377899620414
Top 10 accuracy: 0.25179249261914805
Top 20 accuracy: 0.3262336566849431


In [9]:
# Measure average rank of evidence candidates for every perspectives (rank starts from 1)
total = len(pp_2_ev)
total_rank = 0

for p2e in pp_2_ev:
    gold_ev_id = get_gold_evidence(p2e['pp_id'])
    if gold_ev_id:
        total_rank += p2e['ev_id_list'].index(gold_ev_id) + 1

print("Mean Rank of evidence candidates = {}".format(total_rank / total))

Mean Rank of evidence candidates = 731.7102488401518
